#### Copyright 2019 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# Explorando os embeddings Swivel CORD-19 do TF Hub


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/hub/tutorials/cord_19_embeddings_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/hub/tutorials/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/hub/tutorials/cord_19_embeddings_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
  <td>     <a href="https://tfhub.dev/tensorflow/cord-19/swivel-128d/3"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Ver modelo do TF Hub</a>
</td>
</table>

O módulo de embeddings de texto Swivel CORD-19 do TF Hub (https://tfhub.dev/tensorflow/cord-19/swivel-128d/3) foi criado para ajudar pesquisadores a analisar texto de linguagens naturais relacionado à COVID-19. Esses embeddings foram treinados com títulos, autores, resumos, corpo de textos e títulos de referência de artigos disponíveis no [dataset CORD-19](https://api.semanticscholar.org/CorpusID:216056360).

Neste Colab, vamos:

- Analisar palavras semanticamente similares no espaço de embeddings
- Treinar um classificador com o dataset SciCite usando os embeddings do CORD-19


## Configuração


In [ ]:
import functools
import itertools
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_hub as hub

from tqdm import trange

# Analise os embeddings

Vamos começar analisando o embedding por meio do cálculo e da plotagem de uma matriz de correlação entre os diferentes termos. Se o embedding tiver aprendido a capturar o significado de palavras diferentes com êxito, os vetores de embeddings de palavras semanticamente similares devem estar próximos. Vamos dar uma olhada em alguns termos relacionados à COVID-19.

In [ ]:
# Use the inner product between two embedding vectors as the similarity measure
def plot_correlation(labels, features):
  corr = np.inner(features, features)
  corr /= np.max(corr)
  sns.heatmap(corr, xticklabels=labels, yticklabels=labels)

# Generate embeddings for some terms
queries = [
  # Related viruses
  'coronavirus', 'SARS', 'MERS',
  # Regions
  'Italy', 'Spain', 'Europe',
  # Symptoms
  'cough', 'fever', 'throat'
]

module = hub.load('https://tfhub.dev/tensorflow/cord-19/swivel-128d/3')
embeddings = module(queries)

plot_correlation(queries, embeddings)

Podemos ver que o embedding conseguiu capturar com êxito o significado de diferentes termos. Cada palavra é similar a outras palavras de seu cluster (isto é, "coronavírus" tem alta correlação com "SARS" e "MERS"), embora seja diferente de termos de outros clusters (isto é, a similaridade entre "SARS" e "Spain" – Espanha – é próxima de 0).

Agora, vamos ver como podemos usar esses embeddings para resolver uma tarefa específica.

## SciCite – Classificação da intenção de citação

Esta seção mostra como usar o embedding para tarefas downstream, como classificação de texto. Usaremos o [dataset SciCite](https://www.tensorflow.org/datasets/catalog/scicite) do TensorFlow Datasets para classificar as intenções de citações em artigos acadêmicos. Dada uma frase com uma citação de um artigo acadêmico, vamos classificar se a intenção principal da citação é informação histórica, uso de métodos ou comparação de resultados.

In [ ]:
builder = tfds.builder(name='scicite')
builder.download_and_prepare()
train_data, validation_data, test_data = builder.as_dataset(
    split=('train', 'validation', 'test'),
    as_supervised=True)

In [ ]:
#@title Let's take a look at a few labeled examples from the training set
NUM_EXAMPLES =   10#@param {type:"integer"}

TEXT_FEATURE_NAME = builder.info.supervised_keys[0]
LABEL_NAME = builder.info.supervised_keys[1]

def label2str(numeric_label):
  m = builder.info.features[LABEL_NAME].names
  return m[numeric_label]

data = next(iter(train_data.batch(NUM_EXAMPLES)))


pd.DataFrame({
    TEXT_FEATURE_NAME: [ex.numpy().decode('utf8') for ex in data[0]],
    LABEL_NAME: [label2str(x) for x in data[1]]
})

## Treinamento de um classificador de intenção de citação

Vamos treinar um classificador com o [dataset SciCite](https://www.tensorflow.org/datasets/catalog/scicite) usando o Keras. Vamos criar um modelo que use os embeddings do CORD-19 com uma camada de classificação por cima.

In [ ]:
#@title Hyperparameters { run: "auto" }

EMBEDDING = 'https://tfhub.dev/tensorflow/cord-19/swivel-128d/3'  #@param {type: "string"}
TRAINABLE_MODULE = False  #@param {type: "boolean"}

hub_layer = hub.KerasLayer(EMBEDDING, input_shape=[], 
                           dtype=tf.string, trainable=TRAINABLE_MODULE)

model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(3))
model.summary()
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Treinamento e avaliação do modelo

Vamos treinar e avaliar o modelo para ver o desempenho da tarefa SciCite.

In [ ]:
EPOCHS = 35#@param {type: "integer"}
BATCH_SIZE = 32#@param {type: "integer"}

history = model.fit(train_data.shuffle(10000).batch(BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=validation_data.batch(BATCH_SIZE),
                    verbose=1)

In [ ]:
from matplotlib import pyplot as plt
def display_training_curves(training, validation, title, subplot):
  if subplot%10==1: # set up the subplots on the first call
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    plt.tight_layout()
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

## Avalie o modelo

Vamos conferir o desempenho do modelo. Serão retornados dois valores: perda (um número que representa o erro; quanto menor, melhor) e exatidão.

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print('%s: %.3f' % (name, value))

Podemos ver que a perda diminui rapidamente e que a exatidão aumenta rapidamente. Vamos plotar alguns exemplos para verificar como a previsão está relacionada aos rótulos verdadeiros:

In [ ]:
prediction_dataset = next(iter(test_data.batch(20)))

prediction_texts = [ex.numpy().decode('utf8') for ex in prediction_dataset[0]]
prediction_labels = [label2str(x) for x in prediction_dataset[1]]

predictions = [
    label2str(x) for x in np.argmax(model.predict(prediction_texts), axis=-1)]


pd.DataFrame({
    TEXT_FEATURE_NAME: prediction_texts,
    LABEL_NAME: prediction_labels,
    'prediction': predictions
})

Podemos ver que, para essa amostra aleatória, o modelo prevê o rótulo correto na maioria das vezes, o que indica que ele consegue fazer o embedding de frases científicas muito bem.

# Quais são os próximos passos?

Agora que você conheceu um pouco mais sobre os embeddings Swivel CORD-19 do TF Hub, sugerimos que participe da competição CORD-19 do Kaggle para contribuir com o desenvolvimento de informações científicas a partir de textos acadêmicos relacionados à COVID-19.

- Participe do [Desafio CORD-19 do Kaggle](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge)
- Saiba mais sobre o [COVID-19 Open Research Dataset (CORD-19)](https://api.semanticscholar.org/CorpusID:216056360)
- Confira a documentação e mais detalhes sobre os embeddings do TF Hub em https://tfhub.dev/tensorflow/cord-19/swivel-128d/3
- Explore o espaço de embeddings do CORD-19 com o [TensorFlow Embedding Projector](http://projector.tensorflow.org/?config=https://storage.googleapis.com/tfhub-examples/tensorflow/cord-19/swivel-128d/3/tensorboard/projector_config.json) (projetos de embeddings do TF)